## 数据预览

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys 
from sklearn.model_selection import KFold
%matplotlib inline
# %matplotlib notebook

In [2]:
.\# 解决matplotlib显示中文问题
# 仅适用于Windows
plt.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认字体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
test=pd.read_csv(r'.\\test.csv')
train=pd.read_csv(r'.\\train.csv')
sample=pd.read_csv(r'.\\sample_submission.csv')
test_id=test['id']
combine=[train,test]
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [3]:
train.info()
train.describe()
ps_car_09_cat_train=train['ps_car_09_cat']
ps_car_09_cat_test=test['ps_car_09_cat']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595212 entries, 0 to 595211
Data columns (total 59 columns):
id                595212 non-null int64
target            595212 non-null int64
ps_ind_01         595212 non-null int64
ps_ind_02_cat     595212 non-null int64
ps_ind_03         595212 non-null int64
ps_ind_04_cat     595212 non-null int64
ps_ind_05_cat     595212 non-null int64
ps_ind_06_bin     595212 non-null int64
ps_ind_07_bin     595212 non-null int64
ps_ind_08_bin     595212 non-null int64
ps_ind_09_bin     595212 non-null int64
ps_ind_10_bin     595212 non-null int64
ps_ind_11_bin     595212 non-null int64
ps_ind_12_bin     595212 non-null int64
ps_ind_13_bin     595212 non-null int64
ps_ind_14         595212 non-null int64
ps_ind_15         595212 non-null int64
ps_ind_16_bin     595212 non-null int64
ps_ind_17_bin     595212 non-null int64
ps_ind_18_bin     595212 non-null int64
ps_reg_01         595212 non-null float64
ps_reg_02         595212 non-null float64
ps_re

In [4]:
col_calc=train.columns[train.columns.str[6]=='c']
train.drop(col_calc,axis=1,inplace=True)
test.drop(col_calc,axis=1,inplace=True)

In [5]:
combine=[train,test]

In [6]:
corr=train.corr()

## 每列数据的集合和个数

In [7]:
for data in combine:
    data.drop(['id'],axis=1,inplace=True)
# unique=pd.DataFrame(index=np.arange(0,70482,1),columns=col.index

In [8]:
count=pd.DataFrame(columns=test.columns.values,index=['Count'])
for data in test.columns.values:
    count[data]=len(set(train[data].values))
#     unique[data]=pd.Series(list(set(train[data].values)))
# unique_count=pd.concat([count,unique])
print(count.T['Count'])

ps_ind_01            8
ps_ind_02_cat        5
ps_ind_03           12
ps_ind_04_cat        3
ps_ind_05_cat        8
ps_ind_06_bin        2
ps_ind_07_bin        2
ps_ind_08_bin        2
ps_ind_09_bin        2
ps_ind_10_bin        2
ps_ind_11_bin        2
ps_ind_12_bin        2
ps_ind_13_bin        2
ps_ind_14            5
ps_ind_15           14
ps_ind_16_bin        2
ps_ind_17_bin        2
ps_ind_18_bin        2
ps_reg_01           10
ps_reg_02           19
ps_reg_03         5013
ps_car_01_cat       13
ps_car_02_cat        3
ps_car_03_cat        3
ps_car_04_cat       10
ps_car_05_cat        3
ps_car_06_cat       18
ps_car_07_cat        3
ps_car_08_cat        2
ps_car_09_cat        6
ps_car_10_cat        3
ps_car_11_cat      104
ps_car_11            5
ps_car_12          184
ps_car_13        70482
ps_car_14          850
ps_car_15           15
Name: Count, dtype: int64


## 有缺失值的列和缺失比例

In [9]:
# for data in combine:
#     data.drop(['ps_reg_03','ps_car_05_cat','ps_car_14','ps_car_07_cat'],axis=1,inplace=True)

In [10]:
train[train==-1]=np.nan
train_null=train.isnull().any()
# train_null[train_null==True]

In [11]:
test[test==-1]=np.nan
test_null=test.isnull().any()
# test_null[test_null==True]
combine=[train,test]

In [12]:
train_scale=pd.DataFrame(index=['Train_Scale'],columns=train_null[train_null==True].index)
for index in train_null[train_null==True].index:
    col_nu=train[index].isnull()
    train_scale[index]=len(train[index][col_nu==True])/len(train[index])
train_scale

,ps_ind_02_cat,ps_ind_04_cat,ps_ind_05_cat,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_05_cat,ps_car_07_cat,ps_car_09_cat,ps_car_11,ps_car_12,ps_car_14
Train_Scale,0.000363,0.000139,0.00976,0.181065,0.00018,0.000008,0.690898,0.447825,0.019302,0.000956,0.000008,0.000002,0.071605


In [13]:
test_scale=pd.DataFrame(index=['Test_Scale'],columns=test_null[test_null==True].index)
for index in test_null[test_null==True].index:
    col_nu=test[index].isnull()
    test_scale[index]=len(test[index][col_nu==True])/len(test[index])
test_scale

,ps_ind_02_cat,ps_ind_04_cat,ps_ind_05_cat,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_05_cat,ps_car_07_cat,ps_car_09_cat,ps_car_11,ps_car_14
Test_Scale,0.000344,0.000162,0.009756,0.181094,0.000179,0.000006,0.690972,0.448423,0.019412,0.000982,0.000001,0.071465


## 填充ps_car_09_cat列

In [14]:
corr[abs(corr['ps_car_09_cat'])>0.1]['ps_car_09_cat']

ps_ind_01       -0.102582
ps_car_01_cat    0.272900
ps_car_03_cat    0.272407
ps_car_05_cat   -0.327751
ps_car_09_cat    1.000000
ps_car_11        0.118969
Name: ps_car_09_cat, dtype: float64

In [15]:
guess_car09=[]
len_guess=np.zeros((12))
for data in combine:
    for i in range(0,12):
        guess=data[data['ps_car_01_cat']==i]['ps_car_09_cat'].dropna()
        for j in range(0,5):                   #先根据数字和次数存储起来，找到列中出现最多次数的那个数字
            len_guess[j]=len(guess[guess==j])
        guess_car09.append(len_guess.argmax())
    for i in range(0,12):
        data.loc[(data['ps_car_09_cat'].isnull())&(data['ps_car_01_cat']==i),'ps_car_09_cat']=guess_car09[i]

In [16]:
train['ps_car_09_cat'].fillna(2,inplace=True)
train['ps_car_09_cat'] = train['ps_car_09_cat'].astype(int)
test['ps_car_09_cat'].fillna(2,inplace=True)
test['ps_car_09_cat'] = test['ps_car_09_cat'].astype(int)
combine=[train,test]

### 填充ps_car_03_cat列

In [17]:
corr[abs(corr['ps_car_03_cat'])>0.2]['ps_car_03_cat']

ps_car_03_cat    1.000000
ps_car_05_cat    0.489789
ps_car_08_cat   -0.248662
ps_car_09_cat    0.272407
Name: ps_car_03_cat, dtype: float64

In [18]:
guess_ps03 = np.zeros((2,5))
for data in combine:
    for i in range(0,2):
        for j in range(0,5):
            guess=data[(data['ps_car_08_cat']==i)&(data['ps_car_09_cat']==j)]['ps_car_03_cat'].dropna()
            if len(guess[guess==0])>len(guess[guess==1]):#找到列中出现最多次数的那个数字
                ps_car_03_cat_guess=0
            else:
                ps_car_03_cat_guess=1
            guess_ps03[i][j]=ps_car_03_cat_guess
            
    for i in range(0,2):
        for j in range(0,5):
            data.loc[(data['ps_car_03_cat'].isnull())&(data['ps_car_08_cat']==i)&(data['ps_car_09_cat']==j),'ps_car_03_cat']=guess_ps03[i][j]
combine=[train,test]

### 填充ps_car_05_cat列

In [19]:
corr[abs(corr['ps_car_05_cat'])>0.2]['ps_car_05_cat']

ps_ind_01        0.213009
ps_car_03_cat    0.489789
ps_car_05_cat    1.000000
ps_car_09_cat   -0.327751
Name: ps_car_05_cat, dtype: float64

In [20]:
guess_ps05 = np.zeros((8,5))
for data in combine:
    for i in range(0,8):
        for j in range(0,5):
            guess=data[(data['ps_ind_01']==i)&(data['ps_car_09_cat']==j)]['ps_car_05_cat'].dropna()
            if len(guess[guess==0])>len(guess[guess==1]):#找到列中出现最多次数的那个数字
                ps_car_05_cat_guess=0
            else:
                ps_car_05_cat_guess=1
            guess_ps05[i][j]=ps_car_05_cat_guess
            
    for i in range(0,8):
        for j in range(0,5):
            data.loc[(data['ps_car_05_cat'].isnull())&(data['ps_ind_01']==i)&(data['ps_car_09_cat']==j),'ps_car_05_cat']=guess_ps05[i][j]
combine=[train,test]

### 填充['ps_ind_04_cat']列

In [21]:
corr[abs(corr['ps_ind_04_cat'])>0.1]['ps_ind_04_cat']

ps_ind_02_cat    0.147110
ps_ind_04_cat    1.000000
ps_ind_06_bin   -0.297737
ps_ind_07_bin    0.304949
Name: ps_ind_04_cat, dtype: float64

In [22]:
guess_pic = np.zeros((2,2))
for data in combine:
    for i in range(0,2):
        for j in range(0,2):
            guess=data[(data['ps_ind_06_bin']==i)&(data['ps_ind_07_bin']==j)]['ps_ind_04_cat'].dropna()
            if len(guess[guess==0])>len(guess[guess==1]):#找到列中出现最多次数的那个数字
                ps_ind_04_cat_guess=0
            else:
                ps_ind_04_cat_guess=1
            guess_pic[i][j]=ps_ind_04_cat_guess
            
    for i in range(0,2):
        for j in range(0,2):
            data.loc[(data['ps_ind_04_cat'].isnull())&(data['ps_ind_06_bin']==i)&(data['ps_ind_07_bin']==j),'ps_ind_04_cat']=guess_pic[i][j]
    data['ps_ind_04_cat'] = data['ps_ind_04_cat'].astype(int)
combine=[train,test]

### 填充	ps_ind_02_cat	列

In [23]:
corr[abs(corr['ps_ind_02_cat'])>0.1]['ps_ind_02_cat']

ps_ind_02_cat    1.000000
ps_ind_04_cat    0.147110
ps_ind_06_bin   -0.125653
ps_ind_09_bin    0.110076
ps_car_12       -0.110247
Name: ps_ind_02_cat, dtype: float64

In [24]:
train['ps_ind_04_cat'].values

array([1, 0, 1, ..., 0, 1, 0])

In [25]:
guess_pi02 = np.zeros((2,2))
len_guess=np.zeros((4))
for data in combine:
    for i in range(0,2):
        for j in range(0,2):
            guess=data[(data['ps_ind_04_cat']==i)&(data['ps_ind_06_bin']==j)]['ps_ind_02_cat'].dropna()
            for n in range(0,4):                   #先根据数字和次数存储起来，找到列中出现最多次数的那个数字
                len_guess[n]=len(guess[guess==n])
            guess_pi02[i][j]=len_guess.argmax()+1
            
    for i in range(0,2):
        for j in range(0,2):
            data.loc[(data['ps_ind_02_cat'].isnull())&(data['ps_ind_04_cat']==i)&(data['ps_ind_06_bin']==j),'ps_ind_02_cat']=guess_pi02[i][j]
combine=[train,test]

### 填充ps_ind_05_cat列

In [26]:
corr[abs(corr['ps_ind_05_cat'])>0.05]['ps_ind_05_cat']

ps_ind_03        0.053250
ps_ind_05_cat    1.000000
ps_car_07_cat    0.053777
Name: ps_ind_05_cat, dtype: float64

In [27]:
guess_pi=[]
len_guess=np.zeros((7))
for data in combine:
    for i in range(0,12):
        guess=data[data['ps_ind_03']==i]['ps_ind_05_cat'].dropna()
        for j in range(0,7):                   #先根据数字和次数存储起来，找到列中出现最多次数的那个数字
            len_guess[j]=len(guess[guess==j])
        guess_pi.append(len_guess.argmax())
    for i in range(0,12):
        data.loc[(data['ps_ind_05_cat'].isnull())&(data['ps_ind_03']==i),'ps_ind_05_cat']=guess_pi[i]
    data['ps_ind_05_cat'] = data['ps_ind_05_cat'].astype(int)
combine=[train,test]

### 填充ps_reg_03	列  

In [28]:
corr[abs(corr['ps_reg_03'])>0.1]['ps_reg_03']

ps_ind_01        0.139125
ps_ind_09_bin    0.133761
ps_reg_01        0.637035
ps_reg_02        0.516457
ps_reg_03        1.000000
ps_car_01_cat    0.230339
ps_car_02_cat   -0.135690
Name: ps_reg_03, dtype: float64

In [29]:
#ps_reg_01和ps_reg_02，这里只填充了一部分数据，还有的交叉数据是全部为nan
guess_ps_reg=np.zeros((10,19))
for data in combine:
    for i in range(0,10,1):
        for j in range(0,19,1):
            guess=data[(data['ps_reg_01']==i/10)&(data['ps_reg_02']==j/10)]['ps_reg_03'].dropna()
            guess_ps_reg[i][j]=guess.median()#median()
    for i in range(0,10,1):
        for j in range(0,19,1):
            data.loc[(data['ps_reg_01']==i/10)&(data['ps_reg_02']==j/10)&(data['ps_reg_03'].isnull()),'ps_reg_03']=guess_ps_reg[i][j]
    data['ps_reg_03'] = data['ps_reg_03'].astype(float)
combine=[train,test]

In [30]:
#ps_reg_01和ps_car_01_cat 填充剩下的一部分缺失值
guess_ps_reg=np.zeros((10,12))
for data in combine:
    for i in range(0,10,1):
        for j in range(0,12,1):
            guess=data[(data['ps_reg_01']==i/10)&(data['ps_car_01_cat']==j)]['ps_reg_03'].dropna()
            guess_ps_reg[i][j]=guess.median()#median()
    for i in range(0,10,1):
        for j in range(0,12,1):
            data.loc[(data['ps_reg_01']==i/10)&(data['ps_car_01_cat']==j)&(data['ps_reg_03'].isnull()),'ps_reg_03']=guess_ps_reg[i][j]
    data['ps_reg_03'] = data['ps_reg_03'].astype(float)
combine=[train,test]

In [31]:
train['ps_reg_03_dur']=pd.cut(train['ps_reg_03'],5)
train[['ps_reg_03_dur','target']].groupby(['ps_reg_03_dur'],as_index=False).mean().sort_values(by='ps_reg_03_dur',ascending=True)

,ps_reg_03_dur,target
0,"(0.0573, 0.857]",0.031785
1,"(0.857, 1.652]",0.043147
2,"(1.652, 2.447]",0.052693
3,"(2.447, 3.243]",0.063927
4,"(3.243, 4.038]",0.000000


In [32]:
for data in combine:
    data.loc[data['ps_reg_03']<=0.857,'ps_reg_03']=0
    data.loc[(data['ps_reg_03']>0.857)&(data['ps_reg_03']<=1.652),'ps_reg_03']=1
    data.loc[(data['ps_reg_03']>1.652)&(data['ps_reg_03']<=2.447),'ps_reg_03']=2
    data.loc[(data['ps_reg_03']>2.447)&(data['ps_reg_03']<=3.243),'ps_reg_03']=3
    data.loc[data['ps_reg_03']>3.243,'ps_reg_03']=4
#     data.loc[data['ps_reg_03']>3.375,'ps_reg_03']=5
train.drop(['ps_reg_03_dur'],axis=1,inplace=True)
combine=[train,test]

### 处理缺失列 ps_car_01_cat	

In [33]:
corr[abs(corr['ps_car_01_cat'])>0.1]['ps_car_01_cat']

ps_reg_01        0.184266
ps_reg_02        0.231710
ps_reg_03        0.230339
ps_car_01_cat    1.000000
ps_car_02_cat   -0.152733
ps_car_03_cat    0.138055
ps_car_05_cat   -0.147258
ps_car_09_cat    0.272900
ps_car_11        0.110921
ps_car_12        0.125987
ps_car_13        0.167127
ps_car_15        0.102683
Name: ps_car_01_cat, dtype: float64

In [34]:
#ps_reg_02和ps_car_09_cat
combine=[train,test]
guess_ps_car=np.zeros((10,5))
for data in combine:
    for i in range(0,10,1):
        for j in range(0,5,1):
            len_guess=[]
            guess=data[(data['ps_reg_01']==i/10)&(data['ps_car_09_cat']==j)]['ps_car_01_cat'].dropna()
            for n in range(0,12,1):
                len_guess.append(len(guess[guess==n]))
            guess_ps_car[i][j]=np.array(len_guess).argmax()
    for i in range(0,10,1):
        for j in range(0,5,1):
            data.loc[(data['ps_reg_01']==i/10)&(data['ps_car_09_cat']==j)&(data['ps_car_01_cat'].isnull()),'ps_car_01_cat']=guess_ps_car[i][j]
    data['ps_car_01_cat'].fillna(11,inplace=True)
combine=[train,test]

### 填充ps_car_07_cat列

In [35]:
corr[abs(corr['ps_car_07_cat'])>0.1]['ps_car_07_cat']

ps_car_02_cat    0.103632
ps_car_03_cat   -0.148339
ps_car_04_cat   -0.183374
ps_car_07_cat    1.000000
ps_car_12       -0.125777
ps_car_13       -0.223412
Name: ps_car_07_cat, dtype: float64

In [36]:
guess_pcc=np.zeros((10))
for data in combine:
    for i in range(0,10,1):
        guess=data[data['ps_car_04_cat']==i]['ps_car_07_cat'].dropna()
        if len(guess[guess==0])>len(guess[guess==1]):#找到列中出现最多次数的那个数字
            pcc_guess=0
        else:
            pcc_guess=1
        guess_pcc[i]=pcc_guess
            
    for i in range(0,10,1):
            data.loc[(data['ps_car_04_cat']==i)&(data['ps_car_07_cat'].isnull()),'ps_car_07_cat']=guess_pcc[i]
    data['ps_car_07_cat'] = data['ps_car_07_cat'].astype(int)
combine=[train,test]
train.isnull().any()

target           False
ps_ind_01        False
ps_ind_02_cat    False
ps_ind_03        False
ps_ind_04_cat    False
ps_ind_05_cat    False
ps_ind_06_bin    False
ps_ind_07_bin    False
ps_ind_08_bin    False
ps_ind_09_bin    False
ps_ind_10_bin    False
ps_ind_11_bin    False
ps_ind_12_bin    False
ps_ind_13_bin    False
ps_ind_14        False
ps_ind_15        False
ps_ind_16_bin    False
ps_ind_17_bin    False
ps_ind_18_bin    False
ps_reg_01        False
ps_reg_02        False
ps_reg_03        False
ps_car_01_cat    False
ps_car_02_cat     True
ps_car_03_cat    False
ps_car_04_cat    False
ps_car_05_cat    False
ps_car_06_cat    False
ps_car_07_cat    False
ps_car_08_cat    False
ps_car_09_cat    False
ps_car_10_cat    False
ps_car_11_cat    False
ps_car_11         True
ps_car_12         True
ps_car_13        False
ps_car_14         True
ps_car_15        False
dtype: bool

## 填充ps_car_14列

In [37]:
corr[abs(corr['ps_car_14'])>0.1]['ps_car_14']

ps_car_03_cat   -0.163385
ps_car_08_cat    0.160010
ps_car_14        1.000000
Name: ps_car_14, dtype: float64

In [38]:
guess_car14=[]
for data in combine:
    for i in range(0,2):
        guess=data[data['ps_car_08_cat']==i]['ps_car_14'].dropna()
        guess_car14.append(guess.median())
    for i in range(0,2):
        data.loc[(data['ps_car_14'].isnull())&(data['ps_car_08_cat']==i),'ps_car_14']=guess_car14[i]
    data['ps_car_14'] = data['ps_car_14'].astype(float)
combine=[train,test]

### 删除训练集剩下很少的缺失值ps_car_02_cat和ps_car_12列

In [39]:
train=train.dropna()

In [40]:
test.isnull().any()

ps_ind_01        False
ps_ind_02_cat    False
ps_ind_03        False
ps_ind_04_cat    False
ps_ind_05_cat    False
ps_ind_06_bin    False
ps_ind_07_bin    False
ps_ind_08_bin    False
ps_ind_09_bin    False
ps_ind_10_bin    False
ps_ind_11_bin    False
ps_ind_12_bin    False
ps_ind_13_bin    False
ps_ind_14        False
ps_ind_15        False
ps_ind_16_bin    False
ps_ind_17_bin    False
ps_ind_18_bin    False
ps_reg_01        False
ps_reg_02        False
ps_reg_03        False
ps_car_01_cat    False
ps_car_02_cat     True
ps_car_03_cat    False
ps_car_04_cat    False
ps_car_05_cat    False
ps_car_06_cat    False
ps_car_07_cat    False
ps_car_08_cat    False
ps_car_09_cat    False
ps_car_10_cat    False
ps_car_11_cat    False
ps_car_11         True
ps_car_12        False
ps_car_13        False
ps_car_14        False
ps_car_15        False
dtype: bool

### 填充验证集缺失值很少的ps_car_02_cat

In [41]:
guess=test['ps_car_02_cat'].dropna()
len(guess[guess==0])>len(guess[guess==1])

False

In [42]:
test['ps_car_02_cat'].fillna(1,inplace=True)
test['ps_car_11'].fillna(3,inplace=True)

In [43]:
combine=[train,test]

### 对连续值的列进行归类

In [44]:
# cols_for_dur=['ps_ind_01','ps_reg_02','ps_car_01_cat','ps_reg_01','ps_ind_15',
#               'ps_car_11_cat','ps_car_06_cat','ps_car_12','ps_car_15','ps_ind_03']
# # ['ps_reg_02','ps_ind_15','ps_car_11_cat','ps_car_12','ps_car_15','ps_ind_03']
# for col in cols_for_dur:
#     train['new_col']=pd.cut(train[col],5)
#     list_interval=list(set(train['new_col'].values))
#     list_interval.sort()
#     for data in combine:
#         data.loc[data[col]<list_interval[0].right,col]=0
#         data.loc[(data[col]>list_interval[1].left)&(data[col]<=list_interval[1].right),col]=1
#         data.loc[(data[col]>list_interval[2].left)&(data[col]<=list_interval[2].right),col]=2
#         data.loc[(data[col]>list_interval[3].left)&(data[col]<=list_interval[3].right),col]=3
#         data.loc[data[col]>list_interval[4].left,col]=4
#     train=train.drop(['new_col'],axis=1)
#     combine=[train,test]

In [45]:
# test.head(100000).to_csv(r'c:\Users\ZouJiu\Desktop\front1.csv',index=False)

In [46]:
#类别形整数变量转换为类别变量
col_cato=train.columns[train.columns.str[10]=='c']
def int_to_category(df):
    for col in col_cato:
        df[col]=df[col].astype('category')
for data in combine:
    data=int_to_category(data)
combine=[train,test]





#类别形变量转换为指示性变量
for col in col_cato:
    new_dummies=pd.get_dummies(train[col],prefix=col)
    train=pd.concat([train,new_dummies],axis=1)
    train.drop([col],axis=1,inplace=True)
for col in col_cato:
    new_dummies=pd.get_dummies(test[col],prefix=col)
    test=pd.concat([test,new_dummies],axis=1)
    test.drop([col],axis=1,inplace=True)

len(test.columns)

198

# 交叉验证建模

In [47]:
K=6
from sklearn.model_selection import StratifiedKFold
# kf = KFold(n_splits = K,random_state = 1,shuffle = True)
kf=StratifiedKFold(n_splits = K,random_state = 90,shuffle = True)
X=train.drop(['target'],axis=1)
y=train['target']

In [48]:
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini
def normal_gini(y_true,y_prob):
    return eval_gini(y_true,y_prob)/eval_gini(y_true,y_true)
def xgb_gini(pred,dtrain):
    label=dtrain.get_label()
    return normal_gini(label,pred)

In [49]:
import xgboost as xgb
                        
model=xgb.XGBClassifier(n_estimators=400,
                        max_depth=5,#5
                        objective="binary:logistic",
                        learning_rate=0.07, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.8,
                        scale_pos_weight=1.6,
                        gamma=10,
                        reg_alpha=8,
                        reg_lambda=1.3)
# model=xgb.XGBClassifier(
#     learning_rate = 0.4,
#  n_estimators= 4000,
#  max_depth= 4,
#  min_child_weight= 2,
#  #gamma=1,
#  gamma=0.9,                        
#  subsample=0.8,
#  colsample_bytree=0.8,
#  objective= 'binary:logistic',
#  nthread= -1,
#  scale_pos_weight=1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
import lightgbm as lgb
mode = lgb.LGBMRegressor(objective='regression',
                        num_leaves=200,
                        learning_rate=0.1,
                        n_estimators=40)

In [ ]:
len(X.columns)

In [50]:
y_test_pred=0#0.277
y_valid_pred=0*y
fit_model=[]
for i,(train_index,test_index) in enumerate(kf.split(X,y)):
    y_train,y_valid=y.iloc[train_index],y.iloc[test_index]
    x_train,x_valid=X.iloc[train_index],X.iloc[test_index]
    print('\nFlod:',i)
    fitted_model=model.fit(x_train,y_train)
    pred= fitted_model.predict_proba(x_valid)[:,1]
    y_valid_pred.iloc[test_index]=pred
    print(pd.DataFrame(pred).head(10))
    print('Gini:',eval_gini(y_valid,pred))
    y_test_pred+=fitted_model.predict_proba(test)[:,1]
    fit_model.append(fitted_model)
y_test_pred/=K
print('Gini for full training set:',eval_gini(y,y_valid_pred))
sub=pd.DataFrame({'id':test_id,'target':y_test_pred})
# sub.to_csv(r'.\KFlod_submission.csv',float_format='%.4f',index=False)


Flod: 0
          0
0  0.042941
1  0.029463
2  0.128086
3  0.044702
4  0.054292
5  0.030584
6  0.053039
7  0.049274
8  0.043410
9  0.038646
Gini: 0.286878303843

Flod: 1
          0
0  0.026343
1  0.034946
2  0.033045
3  0.027013
4  0.024676
5  0.052546
6  0.031569
7  0.051490
8  0.045467
9  0.047756
Gini: 0.295890991258

Flod: 2
          0
0  0.048152
1  0.093901
2  0.034687
3  0.053097
4  0.080001
5  0.053997
6  0.041007
7  0.054646
8  0.108539
9  0.058567
Gini: 0.297552728702

Flod: 3
          0
0  0.035558
1  0.025047
2  0.044225
3  0.048113
4  0.040796
5  0.028536
6  0.062147
7  0.044430
8  0.046525
9  0.042051
Gini: 0.287733543471

Flod: 4
          0
0  0.043915
1  0.067342
2  0.051230
3  0.042568
4  0.030792
5  0.062753
6  0.065021
7  0.063634
8  0.033866
9  0.086688
Gini: 0.280825469208

Flod: 5
          0
0  0.038872
1  0.055189
2  0.051140
3  0.081904
4  0.024932
5  0.065624
6  0.061805
7  0.027689
8  0.024524
9  0.036181
Gini: 0.269898555416

Flod: 6
          0
0  0.05

In [51]:
sub.to_csv(r'.\KFlod_submission.csv',float_format='%.6f',index=False)